In [ ]:
'''
In this notebook, we will cover generating radio maps from path gain maps simulated using Ray Tracing. We will load a '.mat' file containing the
simulated emitter path gains and use them to create our own set of radio maps for NN inference.
'''
import torch
import numpy as np
import scipy
import matplotlib.pyplot as plt
from google.colab import drive
drive.mount('/content/gdrive')
%matplotlib inline

In [ ]:
def load_pathgain_maps(data_dir):
    """Return data loader

    Args:
        data_dir: directory to .mat file

    Returns:
        Pathloss Tensor [dB], min pathgain [dB], max pathgain [dB], Latitude Values, Longitude Values, Emitter Latitudes, Emitter Longitudes
    """
    
    f = scipy.io.loadmat(data_dir) # h5py.File(data_dir, 'r')
    pathgain_dB = np.array(f['pathloss'])
    num_maps = pathgain_dB.shape[-1]
    pathgain_dB = torch.tensor(10 * np.log10(pathgain_dB))
    min_pathgain_dB, max_pathgain_dB = pathgain_dB.min(), pathgain_dB.max()

    # MISLABELED LONGITUDE AND LATITUDE VALUES, SWITCHING NAMES
    LONG = np.array(f["LAT"])[:, 0]
    LAT = np.array(f["LONG"])[0]
    emY = np.array(f['emX'])[:, 0]
    emX = np.array(f['emY'])[:, 0]
    return pathgain_dB, min_pathgain_dB, max_pathgain_dB, LAT, LONG, emY, emX

In [ ]:
'''
LOAD THE PATH GAIN MAPS FOR THE TOPOGRAPHICAL CHICAGO REGION
The pathgain Tensor is a 3D tensor indexed as [i_map, iy, ix] and each value is a pathgain for the emitter simulated for that map.
'''
pathgain_dir = "~/chicago_pathlossMultiFile_10052020.mat"

pathgain_dB, min_pathgain_dB, max_pathgain_dB, LAT, LONG, emY, emX = load_pathgain_maps(pathgain_dir)
#  pathgain_dB is a PyTorch Tensor indexed as [i_map, iy, ix] where there are 1000 maps
print("Data Loaded!")

In [ ]:
'''
PLOT SOME EMITTER PATH GAIN MAPS AND VERIFY THE MAP LINES UP WITH THE DOCUMENTED EMITTER LOCATION
The path gain maps contains information for 1000 emitters. We can look at the path gain maps by changing 'i' from 0 to 999.
The emitters were simulated using Forsk Atoll's Aster Model for the E-UTRA 1 DL Band around 2100MHz with a 10MHz Bandwidth using an omnidirectional
transmit antenna. These path gain maps represent a 25.6km x 25.6km topographical region near Chicago.
'''
i = 0
eLat = emY[i]
eLong = emX[i]
e_y = (eLat - LAT[0]) * 512 / (LAT[-1] - LAT[0])
e_x = (eLong - LONG[0]) * 512 / (LONG[-1] - LONG[0])

# PLOT THE PATH GAIN MAP
plt.imshow(torch.flip(pathgain_dB[i], dims=[0]), vmin=min_pathgain_dB, vmax=max_pathgain_dB)
plt.plot([e_x], [512 - e_y], 'rx', label='Emitter Location')
plt.axis('off')
plt.colorbar()
plt.title("Example Path Gain from an Emitter [dB]")
plt.legend()
plt.show()

In [ ]:
'''
GENERATE A POWER MAP AND OCCUPANCY MAP FOR AN EMITTER USING A TRANSMIT POWER
'''
#  Choose the pathgain map (i) and Transmit power (TxPower_W)
i = 1
TxPower_W = 2.  # [0, 2] W
TxPower_dBW = 10 * torch.log10(torch.tensor(TxPower_W))  # dW
TxPower_dBm = TxPower_dBW + 30  # dBm

threshold_dBm = -90  # dBm

power_map = pathgain_dB[i] + TxPower_dBm
occupancy_map = power_map > threshold_dBm

eLat = emY[i]
eLong = emX[i]
e_y = (eLat - LAT[0]) * 512 / (LAT[-1] - LAT[0])
e_x = (eLong - LONG[0]) * 512 / (LONG[-1] - LONG[0])

# PLOT THE POWER MAP
plt.imshow(torch.flip(power_map, dims=[0]))
plt.plot([e_x], [512 - e_y], 'rx', label='Emitter Location')
plt.axis('off')
plt.title("Power Map of the Tx [dBm]")
plt.legend()
plt.colorbar()
plt.show()

# PLOT THE OCCUPANCY MAP
plt.imshow(torch.flip(occupancy_map, dims=[0]), vmin=0, vmax=1, interpolation='nearest')
#plt.plot([e_x], [512 - e_y], 'rx', label='Emitter Location')
plt.axis('off')
plt.title(f"Occupancy Map of the Tx [{threshold_dBm}dBm]")
#plt.legend()
plt.show()

In [ ]:
'''
CREATE A RADIO MAP OF MULTIPLE EMITTERS 
We can create radio maps of multiple emitters assuming the emitters' signals are incoherent. (Such as when base stations over different cells transmit
on different channels)
'''
#  Select a list of pathgain maps (pathgani_indices from 0 to 999) and random Transmit powers (transmit_powers_W)
pathgain_indices = [0, 1, 2]
transmit_powers_W = np.random.uniform(0, 2, size=[len(pathgain_indices)])
threshold_dBm = -90

power_map = torch.zeros(512, 512)
e_xs, e_ys = [], []
print(f"Transmit Powers: {transmit_powers_W} [W]")
fig, axs = plt.subplots(1, 3)
for i_emitter in range(len(pathgain_indices)):
    pathgain_index = pathgain_indices[i_emitter]
    
    eLat = emY[pathgain_index]
    eLong = emX[pathgain_index]
    e_y = (eLat - LAT[0]) * 512 / (LAT[-1] - LAT[0])
    e_x = (eLong - LONG[0]) * 512 / (LONG[-1] - LONG[0])
    e_xs.append(e_x)
    e_ys.append(e_y)
    
    transmit_power_W = transmit_powers_W[i_emitter]
    TxPower_dBW = 10 * torch.log10(torch.tensor(transmit_power_W))
    TxPower_dBm = TxPower_dBW + 30
    
    power_map += 10**((pathgain_dB[pathgain_index] + TxPower_dBm) / 10)  # Convert from dBm to mW to add the power maps together
    axs[i_emitter].imshow(torch.flip(pathgain_dB[pathgain_index] + TxPower_dBm, dims=[0]), vmax=-50,vmin=-140)
    axs[i_emitter].axis("off")
    axs[i_emitter].set_title(f'Emitter: {i_emitter + 1}')
plt.show()

power_map = 10 * torch.log10(power_map)  # Convert the power map back to dBm
occupancy_map = power_map > threshold_dBm

# PLOT THE POWER MAP
plt.imshow(torch.flip(power_map, dims=[0]), vmax=-50,vmin=-140)
plt.plot(e_xs, [512 - e_y for e_y in e_ys], 'rx', label='Emitter Locations')
plt.axis('off')
plt.title("Power Map of the Tx's [dBm]")
plt.legend()
plt.colorbar()
plt.show()

plt.imshow(torch.flip(occupancy_map, dims=[0]), vmin=0, vmax=1, interpolation='nearest')
plt.plot(e_xs, [512 - e_y for e_y in e_ys], 'rx', label='Emitter Locations')
plt.axis('off')
plt.title(f"Occupancy Map of the Tx's [{threshold_dBm}dBm]")
plt.legend()
plt.show()

In [ ]:
def make_power_and_occupancy_maps(pathgain_dB, threshold_dBm, num_maps=10):
    '''
    This function generates a set of power and occupancy with random combinations of emitters and and transmit powers.
    To change the number of emitters possible, edit the lower and upper bound of "num_emitters".
    To change the range of power values in Watts to use, edit the lower and upper bound of "Tx_powers_dBm"
    '''
    num_pathgain_maps = pathgain_dB.size(dim=0)
    num_emitters = np.random.randint(1, 40, size=[num_maps])
    power_maps = torch.zeros(num_maps, 512, 512)

    for i_map in range(num_maps):
        # Pick random emitter path gain indices and Tx powers
        num_emitter = num_emitters[i_map]
        pathgain_indices = np.random.choice(num_pathgain_maps, size=[num_emitter])
        Tx_powers_dBm = np.random.uniform(0, 2, size=[num_emitter])

        # Sum up the power from the individual emitter maps (Need to sum the power in mW, not dBm)
        for i_e, i_pg in enumerate(pathgain_indices):
            power_maps[i_map] += 10**((pathgain_dB[i_pg] + Tx_powers_dBm[i_e] + 30) / 10)  # Have to add power in mW, not dBm
        
    power_maps = 10 * torch.log10(power_maps)
    occupancy_maps = power_maps > threshold_dBm
    return power_maps, occupancy_maps
    

In [ ]:
# GENERATE A RANDOM SET OF RADIO MAPS FOR USE WITH THE OTHER NOTEBOOKS
import os
seed = 423
np.random.seed(seed)
power_maps, occupancy_maps = make_power_and_occupancy_maps(pathgain_dB=pathgain_dB, threshold_dBm=-90, num_maps=10)
torch.save(power_maps, f'{os.getcwd()}/power_maps.pt')


In [ ]:
# PEEK AT THE GENERATED MAPS
i = 6
power_map = power_maps[i]
occupancy_map = occupancy_maps[i]

# PLOT THE POWER MAP
plt.imshow(torch.flip(power_map, dims=[0]))
plt.axis('off')
plt.title("Power Map of the Tx [dBm]")
plt.colorbar()
plt.show()

# PLOT THE OCCUPANCY MAP
plt.imshow(torch.flip(occupancy_map, dims=[0]), vmin=0, vmax=1, interpolation='nearest')
plt.axis('off')
plt.title(f"Occupancy Map of the Tx [{threshold_dBm}dBm]")
plt.show()